In [5]:
from forwardModel2 import *
from SimpleTask import SimpleGridTask
import numpy as np, numpy.random as npr, random as r, SimpleTask
from TransportTask import TransportTask
from NavTask import NavigationTask
import tensorflow as tf
import os
import time


In [30]:
class Henaff_Planning():
    def __init__(self,num_actions,len_action, len_state, iterations, gamma):
        self.x= tf.Variable(tf.random_normal([num_actions,len_action]))
        self.s_0= tf.placeholder(tf.int32, [len_state])
        self.s_f= tf.placeholder(tf.int32, [len_state])
        self.iterations=iterations
        self.num_actions = num_actions
        self.len_action = len_action
        self.len_state = len_state
        

    def optimize(init_state,final_state):
        sess = tf.get_default_session()
        
        #TODO: Need to freeze graph -- https://blog.metaflow.fr/tensorflow-how-to-freeze-a-model-and-serve-it-with-a-python-api-d4f3596b3adc ????
        self.fm=ForwardModel(self.len_state,self.len_state+self.len_action, self.num_actions)
        self.fm.load_model('abcd.ckpt')
        #fm = tf.stop_gradient(fm) not sure how to use stop gradients here 
        
        for i in range(0,self.iterations):
            epsilon=tf.random_normal([self.num_actions,self.len_action])*gamma
            self.x=self.x+epsilon
            self.a=tf.nn.softmax(self.x)
    
            current_state=self.s_0
            c,h = self.fm.getInitialFeatures(1)
            for t in range(0,self.num_actions):
                concat_vec = tf.concat([current_state,self.a[t]])
                current_state,state_out = fm.predict(current_state,c,h)
                c,h = state_out
            
            #TODO: is loss distance loss?? I am doing l2 loss
            loss = tf.reduce_mean(tf.square(current_state - s_f))
            optimize = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)
            _, loss, _ = sess.run([optimize, loss, self.x],{s0:init_state,sf:final_state})
            print("Loss for iteration:" + i,loss)
        #TODO: to convert this into quantized action versions and then return 
        #actions = tf.nn.softmax(self.x)
        
            

In [31]:
with tf.Graph().as_default(), tf.Session() as sess:
    hp = Henaff_Planning(10,10,64,30,0.1)
    #hp.optimize()
    

In [13]:
with tf.Graph().as_default(), tf.Session() as sess:
    fm=ForwardModel(64,64+10, 10, 100)
    fm.load_model('abcd.ckpt')
    #fm = tf.stop_gradient(fm)

INFO:tensorflow:Restoring parameters from abcd.ckpt
